####   Exemple

In [7]:
from Classes_Both.module_marche import DonneeMarche
from Classes_MonteCarlo_LSM.module_brownian import Brownian
from Classes_MonteCarlo_LSM.module_LSM import LSM_method
from Classes_Both.module_option import Option
from Classes_Both.derivatives import OptionDerivatives, OptionDerivativesParameters
import datetime as dt
import numpy as np
import time
from Classes_TrinomialTree.module_arbre_noeud import Arbre
import matplotlib.pyplot as plt
import pandas as pd

### Paramètre

In [8]:
start_date = dt.datetime(2024, 1, 1)
end_date = dt.datetime(2026, 1, 1)

market = DonneeMarche(date_debut= start_date,
volatilite=0.2, 
taux_interet=0.15, 
taux_actualisation=0.15,
# dividends=[{"ex_div_date": dt.datetime(2024, 4, 21), "amount": 3, "rate": 0}], 
dividende_ex_date = dt.datetime(2024, 4, 21),
dividende_montant = 10,
dividende_rate=0,
prix_spot=100)

option = Option(date_pricing=start_date, 
                maturite=end_date, 

                prix_exercice=90, call=False, americaine=True)
pricer = LSM_method(option)

# nombre d'année entre début et fin 
period = (end_date - start_date).days / 365

nb_pas_arbre = 1000
arbre = Arbre(nb_pas_arbre, market, option, pruning = True)
arbre.pricer_arbre()
print(f"Prix arbre option {arbre.prix_option}")

Prix arbre option 3.8750280070050906


### Test solo

In [9]:
nb_pas = int((end_date - start_date).days / 2)
nb_chemin = 10000
seed = 1
method = 'vector'        # vector ou scalar
model_type="polynomial"  # "polynomial" "laguerre" "hermite" "legendre" "chebyshev"
degree_polynome = 2      # utile pour polynomial, laguerre, hermite, legendre, chebyshev

brownian = Brownian(period, nb_pas, nb_chemin, seed)
price, std_error, intervalle = pricer.LSM(brownian, market, method=method, 
                              antithetic=False,poly_degree=degree_polynome, model_type=model_type,
                              print_info = True)
print("Prix: ", price)
        


Nb chemins non antithetic: 10000
Prix min ('non antithetic', 'américaine', 'vector'): 3.6943314654110853
Prix max ('non antithetic', 'américaine', 'vector'): 3.913965932306691
Prix:  3.804148698858888


### Itération nombre pas et chemin

In [10]:
###### test boucle
from itertools import product
liste_chemin = [10000 * i for i in range(1, 11)]
liste_pas_chemin = [(10000 * i, int((end_date - start_date).days / 2)) for i in range(1, 11)]
liste_pas = [10 * i for i in range(1, 31)]
combinations = list(product(liste_chemin,liste_pas))
dico_price = {}

def test_boucle(combi):
    for (path,pas) in combi:
        brownian = Brownian(period, pas, path, 1)
        price, std_error, intervalle = pricer.LSM(brownian, market, method='vector', antithetic=True, 
                                                  poly_degree=2, model_type="polynomial",
                                                  print_info = False)
        if intervalle[0] <= arbre.prix_option <= intervalle[1]:
            if int(arbre.prix_option * 100) / 100 == int(price * 100) / 100:# and std_error < 0.01:
                dico_price[(pas,path)] = {'vrai prix': arbre.prix_option,'price': round(price, 4) ,'ecart-type': round(std_error, 4), 
                'min': round(intervalle[0], 4), 'max':round(intervalle[1], 4), 'proche': "✅    "}
                break
            if int(price * 100) / 100 - 0.01 <= int(arbre.prix_option * 100) / 100 <= int(price * 100) / 100 + 0.01:# and std_error < 0.01:
                dico_price[(pas,path)] = {'vrai prix': arbre.prix_option,'price': round(price, 4) ,'ecart-type': round(std_error, 4), 
                'min': round(intervalle[0], 4), 'max':round(intervalle[1], 4),'proche':'❌    '}
        print(pas,path, price)

test_boucle(combinations)
test_boucle(liste_pas_chemin)
pd.DataFrame(dico_price)

10 10000 3.780097409909457
20 10000 3.798002126499371
30 10000 3.8256137286444383
40 10000 3.821030265763076
50 10000 3.7637662107802403
60 10000 3.7628119153605804
70 10000 3.75448544168616
80 10000 3.8040408815147004
90 10000 3.7928381960720055
100 10000 3.85311602545103
110 10000 3.806965319103551
120 10000 3.800586126822463
130 10000 3.8673031445020833
140 10000 3.83275511291125
150 10000 3.8468478435752855
365 10000 3.8309242721139816
365 20000 3.821660388213067
365 30000 3.810502732896819
365 40000 3.8107631196088914
365 50000 3.8131160157319863
365 60000 3.8136144729503187
365 70000 3.8133788029695745
365 80000 3.812667548098901
365 90000 3.812986885865995
365 100000 3.815481428450533


,160
,10000
vrai prix,3.875028
price,3.8725
ecart-type,0.0292
min,3.8141
max,3.9308
proche,✅


### Grecques

In [11]:
option_deriv = OptionDerivatives(option, market, pricer)  
print("Prix :", option_deriv.price(option_deriv.parameters))
print("Delta :", option_deriv.delta())
print("Vega :", option_deriv.vega())
print("Theta :", option_deriv.theta())
print("Gamma :", option_deriv.gamma())


Prix : (3.8072238479426765, 0.01735516924674396, (3.7725135094491886, 3.8419341864361645))
Delta : -0.31922276412664474
Vega : 34.752301508652295
Theta : -0.08658280319751577
Gamma : 0.021995939175030177
